[オリジナルの英語版はこちら](https://github.com/GoogleCloudPlatform/analytics-componentized-patterns/blob/master/gaming/propensity-model/bqml/bqml_ga4_gaming_propensity_to_churn.ipynb)

In [1]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=Churn%20prediction%20for%20game%20developers%20using%20Google%20Analytics%204%20%28GA4%29%20and%20BigQuery%20ML%20Notebook&download_url=https%3A%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fanalytics-componentized-patterns%2Fmaster%2Fgaming%2Fpropensity-model%2Fbqml%2Fbqml_ga4_gaming_propensity_to_churn.ipynb">
      <img src="https://cloud.google.com/images/products/ai/ai-solutions-icon.svg" alt="AI Platform Notebooks">Run on AI Platform Notebooks</a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/analytics-componentized-patterns/blob/master/gaming/propensity-model/bqml/bqml_ga4_gaming_propensity_to_churn.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

# 概要

このノートブックでは、モバイルゲームアプリのユーザが継続利用するかどうかを判定する機械学習モデルをBigQuery MLを使って訓練、評価、デプロイする方法を示します。
機械学習の訓練データにはGoogle Analytics 4からのアプリの利用ログを利用します。

#### モバイルゲーム業界における傾向分析

10万のモバイルゲームアプリを対象にした[2019 年の調査](https://gameanalytics.com/reports/mobile-gaming-industry-analysis-h1-2019)では、アプリをインストールしてから24時間経過した後にユーザが継続利用している割合は25%しかありません。機械学習を使って1日以内に離脱してしまうユーザの傾向がわかれば、開発者は離脱する可能性の高いユーザに繰り返し使ってもらうための施策を打てるようになります。

ユーザの行動は離脱か定着するかの2つに分類するので、分類問題のアルゴリズムを適用することができます。例えば、線形回帰、XGBoost、ニューラルネットワーク、AutoML Tablesが挙げられます。これらのアルゴリズムはBigQuery MLにて利用することができます。

#### BigQuery ML における傾向モデリング

BigQuery MLを用いると、インフラの準備をする時間を省き、SQLを用いて機械学習モデルの訓練、評価、デプロイを行うことができます。
BigQueryに既に格納されているデータをそのまま訓練に使うことができるので、データの管理上のリスクを軽減することも可能です（訳注：一度データを取り出してしまうと、最新のデータを見ているのか、というような鮮度の問題や、データの漏洩のリスクが出てきます）。

BigQuery MLを使って訓練、デプロイした分類モデルを利用して、ユーザの行動傾向を出力することができます。モデルは0から1の間のスコアを出力します。1に近いほどユーザが離脱する可能性が高く、0であれば定着する可能性が高いことを示しています。

確率スコアを使って、そのまま何もしないと離脱してしまいそうなユーザにターゲットを絞って対策を取ることができます。例えば、何らかのアイテムを付与したり通知を送ったりするなどです。

#### 離脱以外の行動にも応用可能

傾向分析は離脱の可能性のみに限らず、任意の行動に対して行うことができます。例えば、アプリ内決済でアイテムを購入してくれる可能性、ユーザの友達を紹介して一緒に遊んでくれる可能性などです。後者は長期間遊んでくれること、またユーザ数の拡大が見込めます。基本的なワークフローは共通するので、目的に合わせてこのノートブックを改造して使うことができます。

## このノートブックの対象範囲

### データセット

このノートブックでは[公開のBigQuery データセット](https://console.cloud.google.com/bigquery?p=firebase-public-project&d=analytics_153293282&t=events_20181003&page=table)を使います。このデータセットには実際のモバイルゲームアプリから収集されたログデータが含まれます。アプリ名： Flood It! ([Android app](https://play.google.com/store/apps/details?id=com.labpixies.flood), [iOS app](https://itunes.apple.com/us/app/flood-it!/id476943146?mt=8)). [スキーマ](https://support.google.com/analytics/answer/7029846) は Google Analytics for Firebase に由来しますが、 [Google Analytics 4](https://support.google.com/analytics/answer/9358801) と共通します。したがってこのノートブックは Google Analytics for Firebase と Google Analytics 4 のデータに適用可能です。

Google Analytics 4 (GA4) は[イベントベース](https://support.google.com/analytics/answer/9322688) の計測モデルを採用します。イベントはアプリやWebサイトで何が起こったのかについての知見をもたらします。例えば、ユーザの行動、システム側のイベントやエラーの発生です。イベント1件はテーブルの中の1行に対応し、関連する属性値が行の中に入れ子になって格納されています。Google Analytics は標準で多数のイベント種別に対応しますが、それ以外に記録したいイベントを追加することも可能です。

生のイベントデータを直接機械学習モデルの訓練に使うことはできない点に注意してください。このノートブックでは、分類問題を解く機械学習モデルを訓練するために適切なデータ形式へ前処理を行う方法についても扱います。

#### 自身の GA4 データを利用する
（オリジナルの英文にはありますが、本イベントでは公開データを使うため省略します）

#### GA 以外のデータを利用する

（前のセクションと同様、省略します）

### 目標

このノートブックの目標は、GA4のデータとBigQuery MLを利用してユーザの離脱可能性の分析を行う方法を示すことです。
"Flood It!"のデータセットの中から、各ユーザがインストールしてから24時間以内の行動データを取り出し、さまざまな分類モデルを訓練します。

* BigQuery にエクスポートされた Google Analytics 4 データの内容がわかる 
* ユーザの属性、行動、ラベル（離脱する・しない）を使って訓練用データを準備する
* BigQuery ML を使って分類モデルを訓練する
* BigQuery ML を使ってモデルを評価する
* BigQuery ML を使ってユーザが離脱する可能性を推定する

### 費用

このチュートリアルでは下記のサービスの利用料金が発生します。

* BigQuery
* BigQuery ML

料金体系の詳細は[BigQuery](https://cloud.google.com/bigquery/pricing)、[BigQuery ML](https://cloud.google.com/bigquery-ml/pricing)を参照してください。[Pricing
Calculator](https://cloud.google.com/products/calculator/)を使って見積もることもできます。

## セットアップ

### PIP パッケージと依存関係のインストール

In [2]:
!pip install google-cloud-bigquery

In [3]:
# Automatically restart kernel after installs
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)  

{'status': 'ok', 'restart': True}

### Google Cloud プロジェクトの準備

プロジェクトの準備とAPIの有効化は済んでいる想定です。

1. 利用するプロジェクト ID を入力してください。入力後、セルを実行して今後のコマンドを実行するとき Cloud SDK が正しいプロジェクトを利用するようにします。

In [1]:
PROJECT_ID = "qwiklabs-gcp-02-cb5a639b1bac" #replace with your project id
REGION = 'US'

### ライブラリのインポートと定数の定義

In [2]:
from google.cloud import bigquery
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.3f' % x)

### BigQuery データセットの作成

このノートブックでは、`bqmlga4` というデータセットを使います。下記のコマンドを実行して作ります。

In [3]:
DATASET_NAME = "bqmlga4"
!bq mk --location=$REGION --dataset $PROJECT_ID:$DATASET_NAME

Dataset 'qwiklabs-gcp-02-cb5a639b1bac:bqmlga4' successfully created.


## データセット

### Flood it! のイベントデータを利用する

サンプルデータセットは生のイベントデータを持ちます。次のセルを実行して内容を確認してください。

_注_: Jupyter は %%bigquery で始まるセルを SQL クエリとして認識します。

In [4]:
%%bigquery --project $PROJECT_ID

SELECT 
    *
FROM
  `firebase-public-project.analytics_153293282.events_*`
    
TABLESAMPLE SYSTEM (1 PERCENT)

Query is running:   0%|          |

Downloading:   0%|          |

,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,user_properties,user_first_touch_timestamp,user_ltv,device,geo,app_info,traffic_source,stream_id,platform,event_dimensions
0,20180623,1529777737806000,level_complete_quickplay,"[{'key': 'value', 'value': {'string_value': No...",1529731062171000,NaN,<NA>,<NA>,None,E6850F5E0DAB3E339C6A36BDEECCA26F,"[{'key': 'initial_extra_steps', 'value': {'str...",1528480400416000,None,"{'category': 'mobile', 'mobile_brand_name': 'n...","{'continent': 'Americas', 'country': 'United S...","{'id': 'com.labpixies.flood', 'version': '2.62...","{'name': None, 'medium': 'organic', 'source': ...",1051193346,ANDROID,None
1,20180623,1529775402862000,session_start,"[{'key': 'firebase_conversion', 'value': {'str...",1529729812745000,NaN,<NA>,<NA>,None,E6850F5E0DAB3E339C6A36BDEECCA26F,"[{'key': 'initial_extra_steps', 'value': {'str...",1528480400416000,None,"{'category': 'mobile', 'mobile_brand_name': 'n...","{'continent': 'Americas', 'country': 'United S...","{'id': 'com.labpixies.flood', 'version': '2.62...","{'name': None, 'medium': 'organic', 'source': ...",1051193346,ANDROID,None
2,20180623,1529777740802000,session_start,"[{'key': 'firebase_conversion', 'value': {'str...",1529775365862000,NaN,<NA>,<NA>,None,E6850F5E0DAB3E339C6A36BDEECCA26F,"[{'key': 'initial_extra_steps', 'value': {'str...",1528480400416000,None,"{'category': 'mobile', 'mobile_brand_name': 'n...","{'continent': 'Americas', 'country': 'United S...","{'id': 'com.labpixies.flood', 'version': '2.62...","{'name': None, 'medium': 'organic', 'source': ...",1051193346,ANDROID,None
3,20180623,1529775365969001,screen_view,"[{'key': 'firebase_previous_id', 'value': {'st...",1529775355779001,NaN,<NA>,<NA>,None,E6850F5E0DAB3E339C6A36BDEECCA26F,"[{'key': 'initial_extra_steps', 'value': {'str...",1528480400416000,None,"{'category': 'mobile', 'mobile_brand_name': 'n...","{'continent': 'Americas', 'country': 'United S...","{'id': 'com.labpixies.flood', 'version': '2.62...","{'name': None, 'medium': 'organic', 'source': ...",1051193346,ANDROID,None
4,20180623,1529775436143005,screen_view,"[{'key': 'firebase_previous_id', 'value': {'st...",1529775355969005,NaN,<NA>,<NA>,None,E6850F5E0DAB3E339C6A36BDEECCA26F,"[{'key': 'initial_extra_steps', 'value': {'str...",1528480400416000,None,"{'category': 'mobile', 'mobile_brand_name': 'n...","{'continent': 'Americas', 'country': 'United S...","{'id': 'com.labpixies.flood', 'version': '2.62...","{'name': None, 'medium': 'organic', 'source': ...",1051193346,ANDROID,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,20180623,1529798255320000,screen_view,"[{'key': 'firebase_screen_class', 'value': {'s...",1527707813769000,NaN,<NA>,<NA>,None,C6176FAE470C4DDCD0E5BC49DFE2482B,"[{'key': 'first_open_time', 'value': {'string_...",1525229952267000,None,"{'category': 'tablet', 'mobile_brand_name': 'n...","{'continent': 'Americas', 'country': 'United S...","{'id': 'com.google.flood2', 'version': '2.6.30...","{'name': '(direct)', 'medium': '(none)', 'sour...",1051193347,IOS,None
49996,20180623,1529786418635004,app_update,"[{'key': 'previous_app_version', 'value': {'st...",1527087973383004,NaN,<NA>,<NA>,None,C3E24B31253C0BB650F50963F4A7D7E0,"[{'key': 'first_open_time', 'value': {'string_...",1505746073675000,None,"{'category': 'tablet', 'mobile_brand_name': 'n...","{'continent': 'Asia', 'country': 'Japan', 'reg...","{'id': 'com.google.flood2', 'version': '2.6.30...","{'name': '(direct)', 'medium': '(none)', 'sour...",1051193347,IOS,None
49997,20180623,1529786349795005,screen_view,"[{'key': 'firebase_previous_id', 'value': {'st...",1529786332271005,NaN,<NA>,<NA>,None,C3E24B31253C0BB650F50963F4A7D7E0,"[{'key': 'first_open_time', 'value': {'string_...",1505746073675000,None,"{'category': 'tablet', 'mobile_brand_name': 'n...","{'continent': 'Asia', 'country': 'Japan', 

Google Analytics 4 で使われるスキーマを確認できましたか。先に述べたように、 Google Analytics 4 はイベントベースの計測モデルを採用しているので、1イベントは1行に対応します。完全なスキーマの説明や各列の詳細については [サポートページ](https://support.google.com/analytics/answer/7029846) を参照してください。一部の列は入れ子になっていて、下記のような詳細情報が含まれています。

* `app_info`
* `device`
* `ecommerce`
* `event_params`
* `geo`
* `traffic_source`
* `user_properties`

下記のクエリでデータの件数を確認します。ユーザ数は1.5万人、イベントは570万件あることがわかります。

In [5]:
%%bigquery --project $PROJECT_ID

SELECT 
    COUNT(DISTINCT user_pseudo_id) as count_distinct_users,
    COUNT(event_timestamp) as count_events
FROM
  `firebase-public-project.analytics_153293282.events_*`

Query is running:   0%|          |

Downloading:   0%|          |

,count_distinct_users,count_events
0,15175,5700000


### 訓練用データの準備

イベントデータはそのままの形では機械学習モデルの訓練に適した形式にはなっていないので、訓練用データに使うことはできません。このセクションでは、イベントデータを分類モデルの訓練に適した形式に変換します。

ユーザが離脱するかどうかを推定するには、下記のような形式の訓練用データが必要です。 

|User ID|User demographic data|User behavioral data|Churned|
|-|-|-|-|
|User1|(e.g., country, device_type)|(e.g., # of times they did something within a time period)|1
|User2|(e.g., country, device_type)|(e.g., # of times they did something within a time period)|0
|User3|(e.g., country, device_type)|(e.g., # of times they did something within a time period)|1


訓練用データは次のような列を持ちます。

- それぞれの行は一意なユーザIDを持つ
- **ユーザ属性データ** の特徴量
- **行動データ** の特徴量
- 訓練するモデルで推定したい **ラベル** (例: 1 = 離脱する、0 = 定着する)

属性または行動データの片方で訓練することも可能ですが、組み合わせることでより精度の高いモデルになることが期待できます。このセクションでは、二者を組み合わせたデータに変換することにします。

この先では、属性、行動、ラベルのそれぞれについて加工を行い、最後に3つのデータを結合します。

1. ユーザごとのラベルの付与（離脱、定着）
1. ユーザの属性情報の抽出
1. ユーザの行動情報の抽出
1. ラベル、属性、行動データを結合し、訓練データとする

#### ステップ 1: ユーザごとのラベルの付与

データセットそのものにはユーザの「離脱」「定着」を直接示すような特徴量は存在しません。このセクションでは、既存の列を使い、ラベルを作成します。

ユーザの離脱にはさまざまな定義が考えられますが、このノートブックではインストール時から24時間経過した後に使わなくなったことと定義します。

言い換えると、24時間経過後
- ユーザの行動が _記録されていない_ 場合、そのユーザは **離脱した** と判断します。 
- ユーザの行動が _少なくとも1件記録されていれば_ 、ユーザは **定着した** と判断します。

また、アプリを起動してすぐに終了してしまったユーザ（ **直帰** ）も再度利用する可能性は低いと想定されるため、これらのユーザも区別することにします。
直帰したユーザを外し、少なくとも10分はアプリを使ったユーザのデータに絞ってモデルを作ることにします。

これを加味すると **離脱ユーザ** の定義は次のようになります。
> "最低10分はアプリを利用したが、24時間経過後はアプリを利用しなかったユーザ"


生データには、初回（アプリのインストール）から最後の行動までそれぞれのユーザの全てのイベントが保持されているため、SQL を使ってデータを加工し、`churned` と `bounced` のラベルを付与します。

SQL の内容と実行結果を確認してください。

In [6]:
%%bigquery --project $PROJECT_ID 

CREATE OR REPLACE VIEW bqmlga4.returningusers AS (
  WITH firstlasttouch AS (
    SELECT
      user_pseudo_id,
      MIN(event_timestamp) AS user_first_engagement,
      MAX(event_timestamp) AS user_last_engagement
    FROM
      `firebase-public-project.analytics_153293282.events_*`
    WHERE event_name="user_engagement"
    GROUP BY
      user_pseudo_id

  )
  SELECT
    user_pseudo_id,
    user_first_engagement,
    user_last_engagement,
    EXTRACT(MONTH from TIMESTAMP_MICROS(user_first_engagement)) as month,
    EXTRACT(DAYOFYEAR from TIMESTAMP_MICROS(user_first_engagement)) as julianday,
    EXTRACT(DAYOFWEEK from TIMESTAMP_MICROS(user_first_engagement)) as dayofweek,

    #add 24 hr to user's first touch
    (user_first_engagement + 86400000000) AS ts_24hr_after_first_engagement,

#churned = 1 if last_touch within 24 hr of app installation, else 0
IF (user_last_engagement < (user_first_engagement + 86400000000),
    1,
    0 ) AS churned,

#bounced = 1 if last_touch within 10 min, else 0
IF (user_last_engagement <= (user_first_engagement + 600000000),
    1,
    0 ) AS bounced,
  FROM
    firstlasttouch
  GROUP BY
    1,2,3
    );

SELECT 
  * 
FROM 
  bqmlga4.returningusers 
LIMIT 100;

Query is running:   0%|          |

Downloading:   0%|          |

,user_pseudo_id,user_first_engagement,user_last_engagement,month,julianday,dayofweek,ts_24hr_after_first_engagement,churned,bounced
0,2F2B5C88B72A5253AAF448B1E29333F5,1528818508645015,1538232573810027,6,163,3,1528904908645015,0,0
1,38CB6C26C2AA954E22F9EDC61DAEF372,1528991283074002,1530148070034004,6,165,5,1529077683074002,0,0
2,17CCC6417A528996BFB7D538189F62F1,1529017019882001,1529028504171043,6,165,5,1529103419882001,1,0
3,8EDB9A9856CDB5AFC42546106EE46734,1528983800943011,1534963681392018,6,165,5,1529070200943011,0,0
4,8C2CF4B79095AB25D5B6286615DA86A9,1528836405104004,1531556983880164,6,163,3,1528922805104004,0,0
...,...,...,...,...,...,...,...,...,...
95,D45CF251C60626FDB4BB8D50ABB7BA46,1529701919006000,1538406006957078,6,173,6,1529788319006000,0,0
96,5DAD69BB86295E7F73CC8EAA4BFD473A,1531812211399002,1535248619011010,7,198,3,1531898611399002,0,0
97,051BB6E9EC8D2B13A70DB57A132B1A2F,1531835194253005,1532960775419069,7,198,3,1531921594253005,0,0
98,E99D4AD5E5663BABE4AC08C027C775EF,1530589430127001,1531878780302008,7,184,3,1530675830127001,0,0


`churned` 列について: `churned=0` はユーザが24時間経過後もアプリを利用している場合です。それ以外は24時間以内にアプリの操作が終わっているので、`churned=1` になります。

`bounced` 列について: `bounced=1` ユーザの操作が全て10分以内に収まっている場合です。それ以外は `bounced=0` になります。この後、直帰ユーザを訓練用データから外すため、`bounced = 0` の条件を付けてフィルタします。

1.5万人のユーザのうち、どのくらい直帰または定着したのか確認しましょう。

In [7]:
%%bigquery --project $PROJECT_ID

SELECT
    bounced,
    churned, 
    COUNT(churned) as count_users
FROM
    bqmlga4.returningusers
GROUP BY 1,2
ORDER BY bounced

Query is running:   0%|          |

Downloading:   0%|          |

,bounced,churned,count_users
0,0,0,6148
1,0,1,1883
2,1,1,5557


最終的な訓練用データでは `bounced = 0` のユーザのみ使います。5,557 (\~41%) ユーザは10分以内に直帰しました。残りの 8,031 ユーザの中では、 1,883 ユーザ (\~23%) が24時間後には離脱しました。

In [8]:
%%bigquery --project $PROJECT_ID

SELECT
    COUNTIF(churned=1)/COUNT(churned) as churn_rate
FROM
    bqmlga4.returningusers
WHERE bounced = 0

Query is running:   0%|          |

Downloading:   0%|          |

,churn_rate
0,0.234


#### ステップ 2. ユーザの属性情報の抽出

このセクションではそれぞれのユーザの属性情報を抽出します。
データセットにはすでに`app_info`, `device`, `ecommerce`, `event_params`, `geo`のような情報が含まれています。例えば、特定のデバイスや国のユーザが離脱しやすいというような分析ができます。

このノートブックでは、`geo.country`, `device.operating_system`, `device.language` を使います。

一部の属性は途中で変化する可能性があります。例えば、国をまたいだ移動です。ここでは、単純に扱うため、Google Analytics 4 に最初に記録されたときの属性値をそれぞれのユーザの属性とみなします。ユーザ毎に`MIN(event_timestamp)` と集計することでこれは実現します。この変換を行うことで、1ユーザ1行になります。

In [9]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE VIEW bqmlga4.user_demographics AS (

  WITH first_values AS (
      SELECT
          user_pseudo_id,
          geo.country as country,
          device.operating_system as operating_system,
          device.language as language,
          ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp DESC) AS row_num
      FROM `firebase-public-project.analytics_153293282.events_*`
      WHERE event_name="user_engagement"
      )
  SELECT * EXCEPT (row_num)
  FROM first_values
  WHERE row_num = 1
  );

SELECT
  *
FROM
  bqmlga4.user_demographics
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,user_pseudo_id,country,operating_system,language
0,01DADD290826872CB449CB1005A423AB,United States,None,en-us
1,02311BD462580F5DF942A0D6D05ED877,United States,IOS,en-us
2,02508ACF3E0BA4D0C4770C03817AFDE3,Bangladesh,ANDROID,en-us
3,02887D8373139F1D3A223AD62B76CE7A,France,ANDROID,fr-fr
4,04057328B7683ABE11C40CEC50ABF2C1,China,IOS,zh-hans-cn
5,044F3B46E3C2402F9BF4EADCE5C5710A,United States,IOS,en-us
6,050A6530B5D97065581BBD2A9F7F19C5,Canada,ANDROID,en-ca
7,05C3616722D042AB16D2F6FEF2409F3F,United States,IOS,en-us
8,079FE8543EB46F5109283B58A75E8F3D,United States,IOS,en-us
9,0A507AE0DB56197CB90D0FE9439EEC08,United States,ANDROID,en-us


#### ステップ 3. ユーザの行動情報の抽出

1ユーザの行動データは複数のイベントとして記録されているため、行数も複数になります。
これらを集計して、1ユーザ1行にすることがこのセクションの目的です。

どのような行動データを準備すれば、アプリインストールから24時間のデータを元に離脱するか判定できるでしょうか。
`user_first_engagement` からユーザがアプリをインストールした月や曜日を抽出することもできます。

Google Analytics は自動的に [イベント](https://support.google.com/analytics/answer/6317485) を収集します。加えて、[ゲーム向けのイベント](https://support.google.com/analytics/answer/6317494)を使うことが推奨されています。

まずは、どのようなイベントが存在するのかを確認します。`event_name` 列を参照します。

In [10]:
%%bigquery --project $PROJECT_ID

SELECT
    event_name,
    COUNT(event_name) as event_count
FROM
    `firebase-public-project.analytics_153293282.events_*`
GROUP BY 1
ORDER BY
   event_count DESC

Query is running:   0%|          |

Downloading:   0%|          |

,event_name,event_count
0,screen_view,2247623
1,user_engagement,1358958
2,level_start_quickplay,523430
3,level_end_quickplay,349729
4,post_score,242051
5,level_complete_quickplay,191088
6,level_fail_quickplay,137035
7,level_reset_quickplay,122278
8,select_content,105139
9,level_start,74417


このノートブックでは、下記のイベントの発生回数を集計します。

* `user_engagement`
* `level_start_quickplay`
* `level_end_quickplay`
* `level_complete_quickplay`
* `level_reset_quickplay`
* `post_score`
* `spend_virtual_currency`
* `ad_reward`
* `challenge_a_friend`
* `completed_5_levels`
* `use_extra_steps`

SQL を使って、`event_names` ごとの回数をユーザごとに集計します。

In [11]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE VIEW bqmlga4.user_aggregate_behavior AS (
WITH
  events_first24hr AS (
    #select user data only from first 24 hr of using the app
    SELECT
      e.*
    FROM
      `firebase-public-project.analytics_153293282.events_*` e
    JOIN
      bqmlga4.returningusers r
    ON
      e.user_pseudo_id = r.user_pseudo_id
    WHERE
      e.event_timestamp <= r.ts_24hr_after_first_engagement
    )
SELECT
  user_pseudo_id,
  SUM(IF(event_name = 'user_engagement', 1, 0)) AS cnt_user_engagement,
  SUM(IF(event_name = 'level_start_quickplay', 1, 0)) AS cnt_level_start_quickplay,
  SUM(IF(event_name = 'level_end_quickplay', 1, 0)) AS cnt_level_end_quickplay,
  SUM(IF(event_name = 'level_complete_quickplay', 1, 0)) AS cnt_level_complete_quickplay,
  SUM(IF(event_name = 'level_reset_quickplay', 1, 0)) AS cnt_level_reset_quickplay,
  SUM(IF(event_name = 'post_score', 1, 0)) AS cnt_post_score,
  SUM(IF(event_name = 'spend_virtual_currency', 1, 0)) AS cnt_spend_virtual_currency,
  SUM(IF(event_name = 'ad_reward', 1, 0)) AS cnt_ad_reward,
  SUM(IF(event_name = 'challenge_a_friend', 1, 0)) AS cnt_challenge_a_friend,
  SUM(IF(event_name = 'completed_5_levels', 1, 0)) AS cnt_completed_5_levels,
  SUM(IF(event_name = 'use_extra_steps', 1, 0)) AS cnt_use_extra_steps,
FROM
  events_first24hr
GROUP BY
  1
  );

SELECT
  *
FROM
  bqmlga4.user_aggregate_behavior
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,user_pseudo_id,cnt_user_engagement,cnt_level_start_quickplay,cnt_level_end_quickplay,cnt_level_complete_quickplay,cnt_level_reset_quickplay,cnt_post_score,cnt_spend_virtual_currency,cnt_ad_reward,cnt_challenge_a_friend,cnt_completed_5_levels,cnt_use_extra_steps
0,60406ADD3EE12783E9DF7A35EE8570BE,145,19,0,0,10,6,0,0,0,1,0
1,7693A1C0CF271A0BAA670DF090B70EAA,11,3,3,0,0,0,0,0,0,0,0
2,35A64944397A4AFFD30938D599AB8825,7,3,2,2,0,2,0,0,0,0,0
3,7C8A5B2853B828A5756F5BFAB556D6A5,24,0,0,0,0,10,0,0,0,0,0
4,BDE30A70A76C3C9BE423DC53C460239D,6,1,0,0,0,0,0,0,0,0,0
5,4DD9ECDFBC9BDF634A16A484F6094197,9,2,1,1,0,1,0,0,0,0,0
6,F7EB1BF1C86482C138D44D9CB5E4D191,11,3,0,0,2,0,0,0,0,0,0
7,944F8921A3DA4C3E6B913ED257E9FD4C,20,4,4,0,2,0,0,0,0,0,0
8,30B4EA8DC536B0DD556FCCAE4284BD1D,10,0,0,0,0,3,0,0,0,0,0
9,9E82B4A245E78807D6E2B2E6AB6B1AD9,27,7,5,4,0,5,0,0,0,0,0


操作の頻度に加えて、その他の特徴量を加えることもできます。例えば、アプリ内通貨の利用量、アプリ特有のマイルストーン（特定の経験値を達成、レベルアップ回数など）があります。

#### ステップ 4: ラベルと属性、行動データの結合

ステップ1から3までに作成した中間ビューを結合し、訓練用データを作成します。
`bounced = 0` を指定して、直帰したユーザを外している点に注目してください。

In [12]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE VIEW bqmlga4.train AS (
    
  SELECT
    dem.*,
    IFNULL(beh.cnt_user_engagement, 0) AS cnt_user_engagement,
    IFNULL(beh.cnt_level_start_quickplay, 0) AS cnt_level_start_quickplay,
    IFNULL(beh.cnt_level_end_quickplay, 0) AS cnt_level_end_quickplay,
    IFNULL(beh.cnt_level_complete_quickplay, 0) AS cnt_level_complete_quickplay,
    IFNULL(beh.cnt_level_reset_quickplay, 0) AS cnt_level_reset_quickplay,
    IFNULL(beh.cnt_post_score, 0) AS cnt_post_score,
    IFNULL(beh.cnt_spend_virtual_currency, 0) AS cnt_spend_virtual_currency,
    IFNULL(beh.cnt_ad_reward, 0) AS cnt_ad_reward,
    IFNULL(beh.cnt_challenge_a_friend, 0) AS cnt_challenge_a_friend,
    IFNULL(beh.cnt_completed_5_levels, 0) AS cnt_completed_5_levels,
    IFNULL(beh.cnt_use_extra_steps, 0) AS cnt_use_extra_steps,
    ret.user_first_engagement,
    ret.month,
    ret.julianday,
    ret.dayofweek,
    ret.churned
  FROM
    bqmlga4.returningusers ret
  LEFT OUTER JOIN
    bqmlga4.user_demographics dem
  ON 
    ret.user_pseudo_id = dem.user_pseudo_id
  LEFT OUTER JOIN 
    bqmlga4.user_aggregate_behavior beh
  ON
    ret.user_pseudo_id = beh.user_pseudo_id
  WHERE ret.bounced = 0
  );

SELECT
  *
FROM
  bqmlga4.train
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,user_pseudo_id,country,operating_system,language,cnt_user_engagement,cnt_level_start_quickplay,cnt_level_end_quickplay,cnt_level_complete_quickplay,cnt_level_reset_quickplay,cnt_post_score,cnt_spend_virtual_currency,cnt_ad_reward,cnt_challenge_a_friend,cnt_completed_5_levels,cnt_use_extra_steps,user_first_engagement,month,julianday,dayofweek,churned
0,3D339878AE3B0017A1B3313EB2C5A90A,United States,None,en-us,113,0,0,0,0,33,0,0,0,0,0,1528818242527003,6,163,3,0
1,96710B78A294951BF7CDEA5BBEE15766,United States,ANDROID,en-us,14,5,4,4,0,4,0,0,0,0,0,1529263023190003,6,168,1,0
2,073C36044E686CF5D6DEE864559D453F,United States,None,en-us,95,11,8,0,1,8,0,0,0,0,0,1528827608637001,6,163,3,0
3,0C288F38B32B94A75DFE3DF7C3091B63,United States,IOS,en-us,28,10,8,7,0,7,0,0,0,0,0,1528863143354030,6,164,4,0
4,018F03084C88E944163F76A0D0C2FBA1,Spain,ANDROID,de-de,129,64,26,20,0,20,0,0,0,0,0,1528792554849011,6,163,3,0
5,31F8F98A5B838C7375267F10DADBDA24,Saudi Arabia,IOS,en-us,16,9,3,3,4,3,0,0,0,0,0,1529001732755013,6,165,5,0
6,7410C5CC4FA11D31DB5F9B1D8ECD6BD0,Austria,IOS,de-at,11,3,3,2,0,2,0,0,0,0,0,1528906618678010,6,164,4,0
7,5E7FD2E063D2347A72EC52EA919B2550,United States,None,en-us,31,4,3,2,0,2,0,0,0,0,0,1529016280387010,6,165,5,0
8,177581DC2CAB8BCC6A840B7F47A2FEA6,Mexico,IOS,es-us,6,2,1,0,0,0,0,0,0,0,0,1529286457148011,6,169,2,0
9,706FFCFBAA254E9734D74D6493D4CD2D,United States,IOS,en-us,48,5,2,0,0,7,3,0,0,0,0,1529137081803001,6,167,7,0


## BigQuery ML を使ったモデルの訓練

このセクションでは、これまでに準備したデータを BigQuery ML に投入して機械学習モデルを訓練します。

**アルゴリズムの選択**

今回の問題は2値分類なので、一番単純な[線形回帰](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create)から始めます。他にも、[XGBoost](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-boosted-tree), [deep neural networks](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-dnn-models), [AutoML Tables](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-automl) により訓練することも可能です。それぞれのモデルは 0 から 1.0 までのスコアを出力します。

|モデル| model_type| 利点 | 欠点|
|-|-|-|-|
|**線形回帰**| `LOGISTIC_REG` ([ドキュメント](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create))| 他と比べ訓練が速い | 精度はそれほど高くない |
|**XGBoost**| `BOOSTED_TREE_CLASSIFIER` ([ドキュメント](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-boosted-tree))| モデルの性能が高い、特徴量の寄与度を調べることができる | `LOGISTIC_REG` と比べ訓練が遅い|
|**Deep Neural Networks**| `DNN_CLASSIFIER` ([ドキュメント](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-dnn-models))| モデルの性能が良い |  `LOGISTIC_REG` と比べ訓練が遅い|
|**AutoML Tables**| `AUTOML_CLASSIFIER` ([ドキュメント](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-automl))| 非常に性能が良い | 訓練に数時間かかる、モデルの説明性が低い |


**データを訓練用、テスト用に分割する必要はありません**
- `CREATE MODEL` 文を実行すると、 BigQuery ML は自動的にデータを訓練用とテスト用に分割します。従って訓練が終わったときにすぐモデルの評価を行うことが可能です (データの分割を手動で行う方法は[ドキュメント](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create#data_split_method) を参照)。


**ハイパーパラメータチューニング（余力がある人向け）**
機械学習では、ハイパーパラメータと呼ばれる係数をチューニングすることで性能を引き出すことが可能です。
アルゴリズムごとに設定可能なパラメータは異なるので、
[CREATE MODEL のドキュメント](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create)を確認してください。

#### 線形回帰モデルの訓練

下記の SQL を使うと線形回帰モデルの訓練が行えます。1，2分で訓練は終わるはずです。

In [ ]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE MODEL bqmlga4.churn_logreg

OPTIONS(
  ENABLE_GLOBAL_EXPLAIN=TRUE,
  MODEL_TYPE="LOGISTIC_REG",
  INPUT_LABEL_COLS=["churned"]
) AS

SELECT
  *
FROM
  bqmlga4.train

Executing query with job ID: 31cadbb3-8f0d-403f-8397-4b7a57be1d3a
Query executing: 42.08s

#### XGBoost モデル (時間があれば)

（訳注：XGBoost、DNN、AutoML の訓練は時間がかかるため、BigQuery のクエリ画面へノートブックのSQLをコピーし実行するようにしてください）

XGBoost モデルの訓練は数分かかります。

デフォルトのハイパーパラメータはドキュメントを参照してください。
[CREATE MODEL statement for Boosted Tree models using XGBoost](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-boosted-tree)

In [ ]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE MODEL bqmlga4.churn_xgb

OPTIONS(
  ENABLE_GLOBAL_EXPLAIN=TRUE,
  MODEL_TYPE="BOOSTED_TREE_CLASSIFIER",
  INPUT_LABEL_COLS=["churned"]
) AS

SELECT
  * EXCEPT(user_pseudo_id)
FROM
  bqmlga4.train

#### deep neural network (DNN) モデル  (時間があれば)

DNNの訓練も数分かかります。

デフォルトのハイパーパラメータはドキュメントを参照してください。
[CREATE MODEL statement for Deep Neural Network (DNN) models](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-dnn-models)

In [ ]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE MODEL bqmlga4.churn_dnn

OPTIONS(
  ENABLE_GLOBAL_EXPLAIN=TRUE,
  MODEL_TYPE="DNN_CLASSIFIER",
  INPUT_LABEL_COLS=["churned"]
) AS

SELECT
  * EXCEPT(user_pseudo_id)
FROM
  bqmlga4.train

### AutoML Tables モデル (参考)

[AutoML Tables](https://cloud.google.com/automl-tables) は自動的に高性能なモデルを作成する機能です。
AutoML Tables は線形回帰のような単純なモデルから、より高度なアンサンブルなどの手法を用いた複雑、大規模なモデルまで、あらゆる構造のモデルを検証します。

`BUDGET_HOURS` パラメータのデフォルトは 1.0 時間で、最大 72.0 です。実際の処理時間はパラメータに設定した時間より長くなることもあります。

**注:** 訓練には最低でも1時間はかかるので、今回のイベントの時間枠では収まらない可能性が高いです


In [ ]:
# %%bigquery --project $PROJECT_ID

# CREATE OR REPLACE MODEL bqmlga4.churn_automl

# OPTIONS(
#   MODEL_TYPE="AUTOML_CLASSIFIER",
#   INPUT_LABEL_COLS=["churned"],
#   BUDGET_HOURS=1.0
# ) AS

# SELECT
#   * EXCEPT(user_pseudo_id)
# FROM
#   bqmlga4.train

## モデルの評価

モデルを評価するには、訓練したモデルに対して [`ML.EVALUATE`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate) を実行し、メトリクスを得ます。

メトリクスは自動的に分割されたテストデータにより算出されます ([データ分割の詳細](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create#data_split_method))。

（訳注：SQL を使ってノートブック上で評価することもできますが、同じ内容はクラウドコンソールでも表示できます。特に、閾値を変えるのはクラウドコンソールの GUI で操作する方が直感的な操作ができます）

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  *
FROM
  ML.EVALUATE(MODEL bqmlga4.churn_logreg)

`ML.EVALUATE` は `precision`, `recall`, `accuracy`, `f1_score` をデフォルトの閾値 0.5 により算出します。[`THRESHOLD`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate#eval_threshold) パラメータを与えることで閾値は変更可能です。

一般にモデルの性能を比較するには `log_loss` と `roc_auc` が利用されます。

`log_loss` は 0 から 1.0 の間の値を取ります。`log_loss` が0に近ければ、予測されたラベルは実際のラベルに近いことを示します。
`roc_auc` も 0 と 1.0 の間の値を取ります。`roc_auc` が 1.0 に近ければ、モデルは高い識別性能を持つことがわかります。

これらのメトリクスの詳しい定義は [precision and recall](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall), [accuracy](https://developers.google.com/machine-learning/crash-course/classification/accuracy), [f1-score](https://en.wikipedia.org/wiki/F-score), [log_loss](https://en.wikipedia.org/wiki/Loss_functions_for_classification#Logistic_loss) and [roc_auc](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc) を参照してください。

#### 混同行列: 予測と実際の値を比較

モデルを評価するメトリクスに加え、混同行列を用いて予測された値と実際の値の関係を調べます。

行は実際の値、列は予測された値を表します。2値の分類で ML.CONFUSION_MATRIX を実行した結果は下記のような形式になります。

| | Predicted_0 | Predicted_1|
|-|-|-|
|Actual_0| True Negatives | False Positives|
|Actual_1| False Negatives | True Positives|

[混同行列の詳細はこちら](https://developers.google.com/machine-learning/crash-course/classification/true-false-positive-negative)。

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  expected_label,
  _0 AS predicted_0,
  _1 AS predicted_1
FROM
  ML.CONFUSION_MATRIX(MODEL bqmlga4.churn_logreg)

#### ROC カーブ

AUC-ROC カーブは `ML.ROC_CURVE` を用いて描くことができます([ドキュメント](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-roc)).

In [ ]:
%%bigquery df_roc --project $PROJECT_ID
SELECT * FROM ML.ROC_CURVE(MODEL bqmlga4.churn_logreg)

In [ ]:
df_roc

AUC-ROC カーブのプロット

In [ ]:
df_roc.plot(x="false_positive_rate", y="recall", title="AUC-ROC curve")

## モデルを使った推論

[`ML.PREDICT`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict) を使って、離脱する可能性を推定します。次のコードでは `ML.PREDICT` から返ってくる情報を全て表示します。

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  *
FROM
  ML.PREDICT(MODEL bqmlga4.churn_logreg,
  (SELECT * FROM bqmlga4.train)) #本来は訓練用データとは別のテストデータとすべき

一番重要な情報は離脱する可能性だと考えられるので、必要な列のみに絞ります。

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  user_pseudo_id,
  churned,
  predicted_churned,
  predicted_churned_probs[OFFSET(0)].prob as probability_churned
  
FROM
  ML.PREDICT(MODEL bqmlga4.churn_logreg,
  (SELECT * FROM bqmlga4.train)) #can be replaced with a proper test dataset

### 推論した結果を取り出す

##### 結果を BigQuery から直接読み出す

`ML.PREDICT` による結果は、BigQuery Storage API ([ドキュメントとサンプル](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas#download_table_data_using_the_client_library)参照)を使ってPandas のデータフレームとして取り出すことができます。また、別の言語の[BigQuery クライアントライブラリ](https://cloud.google.com/bigquery/docs/reference/libraries)を使うこともできます。

別の方法として、%%bigquery <variable name> を使ってノートブックの Pandas にエクスポートする方法を下記に示します。

In [ ]:
%%bigquery df --project $PROJECT_ID

SELECT
  user_pseudo_id,
  churned,
  predicted_churned,
  predicted_churned_probs[OFFSET(0)].prob as probability_churned
  
FROM
  ML.PREDICT(MODEL bqmlga4.churn_logreg,
  (SELECT * FROM bqmlga4.train)) #can be replaced with a proper test dataset

In [ ]:
df.head()

##### Google Cloud Storage のファイルへ書き出す

Google Cloud Storage (GCS) へエクスポートする方法の中で一番簡単なのはSQL ([ドキュメント](https://cloud.google.com/bigquery/docs/reference/standard-sql/other-statements#export_data_statement))に指定することです。
まずはバケットを作成します。バケット名は全ユーザで一意である必要があるため、今回は一時的に発行されたプロジェクト名をベースにします。

In [ ]:
!gcloud storage buckets create gs://$PROJECT_ID-export

`CREATE TABLE AS SELECT ...` 文を使い、推論した結果を BigQuery のテーブルに保存します。

In [ ]:
%%bigquery --project $PROJECT_ID

CREATE TABLE bqmlga4.predict_logreg AS SELECT
  user_pseudo_id,
  churned,
  predicted_churned,
  predicted_churned_probs[OFFSET(0)].prob as probability_churned
FROM
  ML.PREDICT(MODEL bqmlga4.churn_logreg,
  (SELECT * FROM bqmlga4.train)) #can be replaced with a proper test dataset
;

In [ ]:
import json
query_params = json.dumps({"export_uri": f"gs://{PROJECT_ID}-export/logreg/*"})

In [ ]:
%%bigquery --project $PROJECT_ID --params $query_params

EXPORT DATA OPTIONS (
  uri=@export_uri, 
  format=CSV
) AS 
SELECT
  *
FROM
  bqmlga4.predict_logreg

クラウドコンソールから Cloud Storage を開き、ファイルに書き出されていることを確認してください。

また別の活用方法として、Connected Sheets を使って Google スプレッドシートの画面からデータを参照、利用することも可能です。BigQuery の左側ツリーからテーブルの右にあるメニューを開き、Connected Sheets を選択してください。
Connected Sheets の UI は表計算ソフトですが、その集計は BigQuery のエンジンが使われるため、大規模なデータ処理にも利用可能です。

## 推論した結果の活用

推論した結果を得た後は、ビジネス上の目的にあったさまざまな施策に使うことができます。

この分析では、ユーザを識別するIDとして `user_pseudo_id` を利用しました。実際には、アプリは `user_id` を Google Analytics へ送信すべきです。これにより次のようにファーストパーティデータと組み合わせた活用が可能になります。

* 推論時にファーストパーティデータと結合する
* 推論結果をファーストパーティデータと結合する

結合した後は、

* 推論した結果を Google Analytics へユーザ属性として戻す。これは Google Analytics 4 の[データインポート機能](https://support.google.com/analytics/answer/10071301)によります。
    * 推論した値により、[オーディエンスの作成、変更](https://support.google.com/analytics/answer/2611404) を行い、[オーディエンスターゲティング](https://support.google.com/optimize/answer/6283435)を実施する。 例えば推論値が 0.4 から 0.7 の間のユーザは離脱するか定着するかの境界線上にあるユーザ群と扱う。
* アプリ内の体験を最適化する。Firebase には[セグメントのインポート](https://firebase.google.com/docs/projects/import-segments) 機能があります。 Remote Config, Cloud Messaging, In-App Messaging のような Firebase の機能を使い、ユーザに合わせた体験になるようにします。
* ターゲットを絞ったキャンペーンを CRM ツールを使って実施する。